# Exercice 09 - Nettoyage des donnees (Silver)

## Objectifs
- Comprendre la couche Silver du Data Lake
- Gerer les valeurs nulles
- Supprimer les doublons
- Standardiser les formats de donnees
- Valider la qualite des donnees

---

## 1. La couche Silver

```
+------------------+     +------------------+     +------------------+
|      BRONZE      |     |      SILVER      |     |       GOLD       |
+------------------+     +------------------+     +------------------+
|                  |     |                  |     |                  |
| Donnees brutes   | --> | Donnees propres  | --> | Donnees business |
| Non validees     |     | Validees         |     | Agregatees       |
| Avec doublons    |     | Sans doublons    |     | Optimisees       |
| Formats varies   |     | Formats standards|     | Pretes a l'usage |
|                  |     |                  |     |                  |
+------------------+     +------------------+     +------------------+

Nettoyages Silver :
- Suppression des doublons
- Gestion des valeurs nulles
- Standardisation des formats
- Validation des types
- Correction des erreurs evidentes
```

## 2. Configuration

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, IntegerType, DoubleType, DateType
from datetime import datetime

# Créer la SparkSession
spark = SparkSession.builder \
    .appName("Nettoyage Silver") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0,org.apache.hadoop:hadoop-aws:3.4.1,com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

date_traitement = datetime.now().strftime("%Y-%m-%d")
print(f"Spark prêt - Date : {date_traitement}")

Spark prêt - Date : 2026-01-15


## 3. Creer des donnees de test avec des problemes

In [2]:
# Donnees avec des problemes courants
data_problemes = [
    (1, "Alice", "alice@email.com", "Paris", 25, 50000.0),
    (2, "Bob", "BOB@EMAIL.COM", "paris", 30, 60000.0),      # Email en majuscules, ville minuscule
    (3, "Charlie", None, "Lyon", 35, None),                  # Valeurs nulles
    (4, "Diana", "diana@email.com", "  Marseille  ", -5, 70000.0),  # Espaces, age negatif
    (1, "Alice", "alice@email.com", "Paris", 25, 50000.0),   # Doublon
    (5, "Eve", "eve@email", "Toulouse", 28, 55000.0),        # Email invalide
    (6, "Frank", "frank@email.com", "", 40, 80000.0),        # Ville vide
    (7, "Grace", "grace@email.com", "Nice", 150, 65000.0),   # Age impossible
]

colonnes = ["id", "nom", "email", "ville", "age", "salaire"]
df_bronze = spark.createDataFrame(data_problemes, colonnes)

print("Donnees brutes (Bronze) :")
df_bronze.show(truncate=False)

Donnees brutes (Bronze) :
+---+-------+---------------+-------------+---+-------+
|id |nom    |email          |ville        |age|salaire|
+---+-------+---------------+-------------+---+-------+
|1  |Alice  |alice@email.com|Paris        |25 |50000.0|
|2  |Bob    |BOB@EMAIL.COM  |paris        |30 |60000.0|
|3  |Charlie|NULL           |Lyon         |35 |NULL   |
|4  |Diana  |diana@email.com|  Marseille  |-5 |70000.0|
|1  |Alice  |alice@email.com|Paris        |25 |50000.0|
|5  |Eve    |eve@email      |Toulouse     |28 |55000.0|
|6  |Frank  |frank@email.com|             |40 |80000.0|
|7  |Grace  |grace@email.com|Nice         |150|65000.0|
+---+-------+---------------+-------------+---+-------+



## 4. Supprimer les doublons

In [3]:
# Compter avant
print(f"Lignes avant : {df_bronze.count()}")

# Supprimer les doublons exacts
df_sans_doublons = df_bronze.dropDuplicates()
print(f"Lignes apres dropDuplicates() : {df_sans_doublons.count()}")

# Supprimer les doublons sur certaines colonnes
df_sans_doublons_id = df_bronze.dropDuplicates(["id"])
print(f"Lignes apres dropDuplicates(['id']) : {df_sans_doublons_id.count()}")

Lignes avant : 8
Lignes apres dropDuplicates() : 7
Lignes apres dropDuplicates(['id']) : 7


In [4]:
# Continuer avec les donnees sans doublons
df = df_sans_doublons
df.show()

+---+-------+---------------+-------------+---+-------+
| id|    nom|          email|        ville|age|salaire|
+---+-------+---------------+-------------+---+-------+
|  1|  Alice|alice@email.com|        Paris| 25|50000.0|
|  2|    Bob|  BOB@EMAIL.COM|        paris| 30|60000.0|
|  4|  Diana|diana@email.com|  Marseille  | -5|70000.0|
|  5|    Eve|      eve@email|     Toulouse| 28|55000.0|
|  6|  Frank|frank@email.com|             | 40|80000.0|
|  7|  Grace|grace@email.com|         Nice|150|65000.0|
|  3|Charlie|           NULL|         Lyon| 35|   NULL|
+---+-------+---------------+-------------+---+-------+



## 5. Gerer les valeurs nulles

In [5]:
# Compter les valeurs nulles par colonne
print("Valeurs nulles par colonne :")
for col in df.columns:
    nb_nulls = df.filter(F.col(col).isNull()).count()
    print(f"  {col}: {nb_nulls}")

Valeurs nulles par colonne :
  id: 0
  nom: 0
  email: 1
  ville: 0
  age: 0
  salaire: 1


In [6]:
# Option 1 : Supprimer les lignes avec des nulls
df_drop_na = df.dropna()
print(f"Apres dropna() : {df_drop_na.count()} lignes")

Apres dropna() : 6 lignes


In [7]:
# Option 2 : Remplacer les nulls par des valeurs par defaut
df_fill = df.fillna({
    "email": "inconnu@example.com",
    "salaire": 0.0
})

df_fill.show()

+---+-------+-------------------+-------------+---+-------+
| id|    nom|              email|        ville|age|salaire|
+---+-------+-------------------+-------------+---+-------+
|  1|  Alice|    alice@email.com|        Paris| 25|50000.0|
|  2|    Bob|      BOB@EMAIL.COM|        paris| 30|60000.0|
|  4|  Diana|    diana@email.com|  Marseille  | -5|70000.0|
|  5|    Eve|          eve@email|     Toulouse| 28|55000.0|
|  6|  Frank|    frank@email.com|             | 40|80000.0|
|  7|  Grace|    grace@email.com|         Nice|150|65000.0|
|  3|Charlie|inconnu@example.com|         Lyon| 35|    0.0|
+---+-------+-------------------+-------------+---+-------+



In [8]:
# Option 3 : Remplacer par la moyenne (pour les numeriques)
moyenne_salaire = df.agg(F.avg("salaire")).collect()[0][0]
print(f"Salaire moyen : {moyenne_salaire}")

df_fill_avg = df.fillna({"salaire": moyenne_salaire})
df_fill_avg.show()

Salaire moyen : 63333.333333333336
+---+-------+---------------+-------------+---+------------------+
| id|    nom|          email|        ville|age|           salaire|
+---+-------+---------------+-------------+---+------------------+
|  1|  Alice|alice@email.com|        Paris| 25|           50000.0|
|  2|    Bob|  BOB@EMAIL.COM|        paris| 30|           60000.0|
|  4|  Diana|diana@email.com|  Marseille  | -5|           70000.0|
|  5|    Eve|      eve@email|     Toulouse| 28|           55000.0|
|  6|  Frank|frank@email.com|             | 40|           80000.0|
|  7|  Grace|grace@email.com|         Nice|150|           65000.0|
|  3|Charlie|           NULL|         Lyon| 35|63333.333333333336|
+---+-------+---------------+-------------+---+------------------+



## 6. Standardiser les formats

In [9]:
# Appliquer plusieurs nettoyages
df_clean = df_fill \
    .withColumn("email", F.lower(F.col("email"))) \
    .withColumn("ville", F.trim(F.col("ville"))) \
    .withColumn("ville", F.initcap(F.col("ville"))) \
    .withColumn("nom", F.initcap(F.col("nom")))

print("Apres standardisation :")
df_clean.show(truncate=False)

Apres standardisation :
+---+-------+-------------------+---------+---+-------+
|id |nom    |email              |ville    |age|salaire|
+---+-------+-------------------+---------+---+-------+
|1  |Alice  |alice@email.com    |Paris    |25 |50000.0|
|2  |Bob    |bob@email.com      |Paris    |30 |60000.0|
|4  |Diana  |diana@email.com    |Marseille|-5 |70000.0|
|5  |Eve    |eve@email          |Toulouse |28 |55000.0|
|6  |Frank  |frank@email.com    |         |40 |80000.0|
|7  |Grace  |grace@email.com    |Nice     |150|65000.0|
|3  |Charlie|inconnu@example.com|Lyon     |35 |0.0    |
+---+-------+-------------------+---------+---+-------+



In [10]:
# Traiter les villes vides
df_clean = df_clean.withColumn(
    "ville",
    F.when(F.col("ville") == "", "Inconnue").otherwise(F.col("ville"))
)

df_clean.show(truncate=False)

+---+-------+-------------------+---------+---+-------+
|id |nom    |email              |ville    |age|salaire|
+---+-------+-------------------+---------+---+-------+
|1  |Alice  |alice@email.com    |Paris    |25 |50000.0|
|2  |Bob    |bob@email.com      |Paris    |30 |60000.0|
|4  |Diana  |diana@email.com    |Marseille|-5 |70000.0|
|5  |Eve    |eve@email          |Toulouse |28 |55000.0|
|6  |Frank  |frank@email.com    |Inconnue |40 |80000.0|
|7  |Grace  |grace@email.com    |Nice     |150|65000.0|
|3  |Charlie|inconnu@example.com|Lyon     |35 |0.0    |
+---+-------+-------------------+---------+---+-------+



## 7. Valider les donnees

In [11]:
# Valider l'age (entre 0 et 120)
df_valid = df_clean.withColumn(
    "age_valide",
    F.when((F.col("age") >= 0) & (F.col("age") <= 120), True).otherwise(False)
)

print("Ages invalides :")
df_valid.filter(F.col("age_valide") == False).show()

Ages invalides :
+---+-----+---------------+---------+---+-------+----------+
| id|  nom|          email|    ville|age|salaire|age_valide|
+---+-----+---------------+---------+---+-------+----------+
|  4|Diana|diana@email.com|Marseille| -5|70000.0|     false|
|  7|Grace|grace@email.com|     Nice|150|65000.0|     false|
+---+-----+---------------+---------+---+-------+----------+



In [12]:
# Corriger ou filtrer les ages invalides
df_age_corrige = df_clean.withColumn(
    "age",
    F.when(F.col("age") < 0, None)
     .when(F.col("age") > 120, None)
     .otherwise(F.col("age"))
)

df_age_corrige.show()

+---+-------+-------------------+---------+----+-------+
| id|    nom|              email|    ville| age|salaire|
+---+-------+-------------------+---------+----+-------+
|  1|  Alice|    alice@email.com|    Paris|  25|50000.0|
|  2|    Bob|      bob@email.com|    Paris|  30|60000.0|
|  4|  Diana|    diana@email.com|Marseille|NULL|70000.0|
|  5|    Eve|          eve@email| Toulouse|  28|55000.0|
|  6|  Frank|    frank@email.com| Inconnue|  40|80000.0|
|  7|  Grace|    grace@email.com|     Nice|NULL|65000.0|
|  3|Charlie|inconnu@example.com|     Lyon|  35|    0.0|
+---+-------+-------------------+---------+----+-------+



In [13]:
# Valider le format email avec regex
pattern_email = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

df_email_valide = df_age_corrige.withColumn(
    "email_valide",
    F.col("email").rlike(pattern_email)
)

print("Emails invalides :")
df_email_valide.filter(F.col("email_valide") == False).show(truncate=False)

Emails invalides :
+---+---+---------+--------+---+-------+------------+
|id |nom|email    |ville   |age|salaire|email_valide|
+---+---+---------+--------+---+-------+------------+
|5  |Eve|eve@email|Toulouse|28 |55000.0|false       |
+---+---+---------+--------+---+-------+------------+



## 8. Pipeline de nettoyage complet

In [14]:
def nettoyer_donnees(df):
    """
    Pipeline de nettoyage complet.
    
    Args:
        df: DataFrame brut (Bronze)
    
    Returns:
        DataFrame nettoye (Silver)
    """
    # 1. Supprimer les doublons
    df = df.dropDuplicates(["id"])
    
    # 2. Gerer les nulls
    df = df.fillna({
        "email": "inconnu@example.com",
        "ville": "Inconnue",
        "salaire": 0.0
    })
    
    # 3. Standardiser les formats
    df = df.withColumn("email", F.lower(F.trim(F.col("email")))) \
           .withColumn("ville", F.initcap(F.trim(F.col("ville")))) \
           .withColumn("nom", F.initcap(F.trim(F.col("nom"))))
    
    # 4. Traiter les villes vides
    df = df.withColumn(
        "ville",
        F.when(F.col("ville") == "", "Inconnue").otherwise(F.col("ville"))
    )
    
    # 5. Valider et corriger l'age
    df = df.withColumn(
        "age",
        F.when((F.col("age") >= 0) & (F.col("age") <= 120), F.col("age"))
         .otherwise(None)
    )
    
    # 6. Ajouter les metadonnees
    df = df.withColumn("_cleaned_date", F.lit(datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
    
    return df

In [15]:
# Appliquer le pipeline
df_silver = nettoyer_donnees(df_bronze)

print("Donnees nettoyees (Silver) :")
df_silver.show(truncate=False)

Donnees nettoyees (Silver) :
+---+-------+-------------------+---------+----+-------+-------------------+
|id |nom    |email              |ville    |age |salaire|_cleaned_date      |
+---+-------+-------------------+---------+----+-------+-------------------+
|1  |Alice  |alice@email.com    |Paris    |25  |50000.0|2026-01-15 16:02:11|
|2  |Bob    |bob@email.com      |Paris    |30  |60000.0|2026-01-15 16:02:11|
|3  |Charlie|inconnu@example.com|Lyon     |35  |0.0    |2026-01-15 16:02:11|
|4  |Diana  |diana@email.com    |Marseille|NULL|70000.0|2026-01-15 16:02:11|
|5  |Eve    |eve@email          |Toulouse |28  |55000.0|2026-01-15 16:02:11|
|6  |Frank  |frank@email.com    |Inconnue |40  |80000.0|2026-01-15 16:02:11|
|7  |Grace  |grace@email.com    |Nice     |NULL|65000.0|2026-01-15 16:02:11|
+---+-------+-------------------+---------+----+-------+-------------------+



## 9. Rapport de qualite

In [16]:
def rapport_qualite(df_avant, df_apres):
    """
    Genere un rapport de qualite des donnees.
    """
    print("=" * 50)
    print("RAPPORT DE QUALITE")
    print("=" * 50)
    
    print(f"\nLignes avant  : {df_avant.count()}")
    print(f"Lignes apres  : {df_apres.count()}")
    print(f"Lignes retirees: {df_avant.count() - df_apres.count()}")
    
    print("\nValeurs nulles (apres nettoyage) :")
    for col in df_apres.columns:
        if not col.startswith("_"):
            nb_nulls = df_apres.filter(F.col(col).isNull()).count()
            pct = (nb_nulls / df_apres.count()) * 100 if df_apres.count() > 0 else 0
            print(f"  {col}: {nb_nulls} ({pct:.1f}%)")
    
    print("=" * 50)

rapport_qualite(df_bronze, df_silver)

RAPPORT DE QUALITE

Lignes avant  : 8
Lignes apres  : 7
Lignes retirees: 1

Valeurs nulles (apres nettoyage) :
  id: 0 (0.0%)
  nom: 0 (0.0%)
  email: 0 (0.0%)
  ville: 0 (0.0%)
  age: 2 (28.6%)
  salaire: 0 (0.0%)


## 10. Sauvegarder dans Silver

In [17]:
# Sauvegarder les donnees nettoyees
chemin_silver = f"s3a://silver/users/{date_traitement}"
df_silver.write.mode("overwrite").parquet(chemin_silver)

print(f"Sauvegarde Silver : {chemin_silver}")

Sauvegarde Silver : s3a://silver/users/2026-01-15


---

## Exercice

**Objectif** : Nettoyer les donnees Northwind customers

**Consigne** :
1. Lisez la table `customers` depuis PostgreSQL ou Bronze
2. Appliquez les nettoyages suivants :
   - Standardisez les noms de pays (majuscules)
   - Supprimez les espaces en debut/fin de texte
   - Gerez les valeurs nulles
3. Sauvegardez dans Silver

A vous de jouer :

In [18]:
# TODO: Lire les customers depuis PostgreSQL
jdbc_url = "jdbc:postgresql://postgres:5432/app"
jdbc_properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

df_customers = spark.read.jdbc(url=jdbc_url, table="customers", properties=jdbc_properties)
print(f"Nombre de clients chargés : {df_customers.count()}")
df_customers.show(5)

Nombre de clients chargés : 91
+-----------+--------------------+------------------+--------------------+--------------------+-----------+------+-----------+-------+--------------+--------------+
|customer_id|        company_name|      contact_name|       contact_title|             address|       city|region|postal_code|country|         phone|           fax|
+-----------+--------------------+------------------+--------------------+--------------------+-----------+------+-----------+-------+--------------+--------------+
|      ALFKI| Alfreds Futterkiste|      Maria Anders|Sales Representative|       Obere Str. 57|     Berlin|  NULL|      12209|Germany|   030-0074321|   030-0076545|
|      ANATR|Ana Trujillo Empa...|      Ana Trujillo|               Owner|Avda. de la Const...|México D.F.|  NULL|      05021| Mexico|  (5) 555-4729|  (5) 555-3745|
|      ANTON|Antonio Moreno Ta...|    Antonio Moreno|               Owner|     Mataderos  2312|México D.F.|  NULL|      05023| Mexico|  (5) 555-

In [19]:
# TODO: Appliquer les nettoyages
from pyspark.sql import functions as F

# Nettoyage de base sur les strings
df_clean = df_customers
for col_name, dtype in df_customers.dtypes:
    if dtype == "string":
        df_clean = df_clean.withColumn(col_name, F.trim(F.col(col_name)))

# Standardisation et gestion des NULLs
df_silver_customers = df_clean \
    .withColumn("country", F.upper(F.col("country"))) \
    .withColumn("city", F.initcap(F.col("city"))) \
    .fillna({
        "region": "Non spécifié",
        "fax": "Non renseigné"
    }) \
    .dropDuplicates(["customer_id"])

print("Aperçu des données nettoyées :")
df_silver_customers.select("customer_id", "company_name", "city", "country", "region").show(5)

Aperçu des données nettoyées :
+-----------+--------------------+-----------+-------+------------+
|customer_id|        company_name|       city|country|      region|
+-----------+--------------------+-----------+-------+------------+
|      ALFKI| Alfreds Futterkiste|     Berlin|GERMANY|Non spécifié|
|      ANATR|Ana Trujillo Empa...|México D.f.| MEXICO|Non spécifié|
|      ANTON|Antonio Moreno Ta...|México D.f.| MEXICO|Non spécifié|
|      AROUT|     Around the Horn|     London|     UK|Non spécifié|
|      BERGS|  Berglunds snabbköp|      Luleå| SWEDEN|Non spécifié|
+-----------+--------------------+-----------+-------+------------+
only showing top 5 rows


In [20]:
# TODO: Sauvegarder dans Silver
from datetime import datetime

date_traitement = datetime.now().strftime("%Y-%m-%d")
chemin_silver = f"s3a://silver/customers/{date_traitement}"

df_final = df_silver_customers.withColumn("_cleaned_at", F.current_timestamp())

# Écriture
df_final.write.mode("overwrite").parquet(chemin_silver)

print(f"Sauvegarde Silver terminée avec succès : {chemin_silver}")
print(f"Nombre de clients sauvegardés : {df_final.count()}")

Sauvegarde Silver terminée avec succès : s3a://silver/customers/2026-01-15
Nombre de clients sauvegardés : 91


---

## Resume

Dans ce notebook, vous avez appris :
- Le role de la couche **Silver** dans un Data Lake
- Comment **supprimer les doublons** avec dropDuplicates()
- Comment **gerer les valeurs nulles** avec dropna() et fillna()
- Comment **standardiser les formats** (trim, lower, initcap)
- Comment **valider les donnees** avec des regles metier
- Comment creer un **pipeline de nettoyage** reutilisable

### Prochaine etape
Dans le prochain notebook, nous apprendrons les transformations avancees pour la couche Silver.